In [1]:
import os
import sys
import DifferentiableStateSpaceModels as DSSM
from DifferentiableStateSpaceModels import *
from sympy import *
from IPython.display import display, Math, Latex

init_printing()  # for nice sympy expressions in jupyter
sys.path.append(DSSM.default_model_cache_location())

# RBC Example

In [2]:
# Define the symbols
α, β, ρ, δ, σ, Ω_1 = symbols('α β ρ δ σ Ω_1')  # parameters
t = symbols('t', integer=True)  # time index (integer)
k, z, c, q = symbols('k z c q', cls=Function)  # undefined functions

# Define the states, controls, and parameters
x = [k, z]  # states - list of functions
y = [c, q]  # controls - list of functions
p = [α, β, ρ, δ, σ, Ω_1]  # parameters - list of symbols

# Define the system of model equations - list of expressions
H = [
    1 / c(t) - (β / c(t + 1)) * (α * exp(z(t + 1)) * k(t + 1)**(α - 1) + (1 - δ)),
    c(t) + k(t + 1) - (1 - δ) * k(t) - q(t),
    q(t) - exp(z(t)) * k(t)**α,
    z(t + 1) - ρ * z(t)
]

# Define the steady states - list of equations
steady_states = [
    Eq(k(oo), (((1 / β) - 1 + δ) / α)**(1 / (α - 1))),
    Eq(z(oo), 0),
    Eq(c(oo), (((1 / β) - 1 + δ) / α)**(α / (α - 1)) - δ * (((1 / β) - 1 + δ) / α)**(1 / (α - 1))),
    Eq(q(oo), (((1 / β) - 1 + δ) / α)**(α / (α - 1)))
]

# Define the matrix for the 1 shock
Γ = Matrix([σ])  # DenseMatrix

# Define the n_x * n_ϵ matrix
η = Matrix([0, -1])  # DenseMatrix

# Define the observation matrix
Q = Matrix([[1.0, 0, 0, 0], [0, 0, 1.0, 0]])  # DenseMatrix

# Define the diagonal cholesky of covariance matrix for observation noise
Ω = [Ω_1, Ω_1]  # list of symbols

# Generates the files and includes if required.  If the model is already created, then just loads
overwrite_model_cache = True

In [3]:
# Create the model
model_rbc = DSSM.make_and_include_perturbation_model(model_name='rbc', H=H, t=t, y=y, x=x, steady_states=steady_states, p=p, Γ=Γ, η=η, Q=Q, Ω=Ω, overwrite_model_cache=overwrite_model_cache)
print(model_rbc)

Perturbation Model: n_y = 2, n_x = 2, n_p = 6, n_ϵ = 1, n_z = 2
 y = ['c', 'q'] 
 x = ['k', 'z'] 
 p = ['α', 'β', 'ρ', 'δ', 'σ', 'Ω_1']


In [4]:
display(Math(model_rbc.model_H_latex()))

<IPython.core.display.Math object>

In [5]:
display(Math(model_rbc.model_steady_states_latex()))

<IPython.core.display.Math object>

# Solving Perturbations

In [6]:
p_f = {'ρ': 0.2, 'δ': 0.02, 'σ': 0.01, 'Ω_1': 0.01}  # Fixed parameters
p_d = {'α': 0.5, 'β': 0.95}  # Pseudo-true values
m = model_rbc

In [12]:
list(p_d.values())